# Designing 3D CNNs

The dimension of a CNN does not refer to the dimensions of the input but to the dimensions of the kernel stride.  
1D kernel moves only left-right (or up-down)  
2D kernel moves left-right and up-down  
3D kernel moves left-right, up-down and forward-backwards.   

Thus with a kernel of size (3,3,20) a 3D volume of size (150,150,20) could be processed. The present 2D CNN from pytorch and fastai could thus easily be adapted. However, small findings which only occur in a feq slices could disappear in the convolutions, so 3D CNNs with smaller kernels might be better.  



In [5]:
# default_exp models
# export 

import torchvision
from torch import nn

## Adapting 2d CNNs

In [20]:
import fastai.vision.models as models
from fastai.vision import *

In [30]:
torchvision.models.resnet18??

Signature: torchvision.models.resnet18(pretrained=False, progress=True, **kwargs)
Source:   
def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)
File:      ~/anaconda3/envs/fastai-v2/lib/python3.7/site-packages/torchvision/models/resnet.py
Type:      function


In [6]:
# export
# https://github.com/Ontheroad123/ImageNet/blob/master/torch-alexnet-3D.py

class AlexNet_3D(nn.Module):

    def __init__(self, num_classes=2, n_dim=7):
        super(AlexNet_3D, self).__init__()
        self.features = nn.Sequential(
            nn.Conv3d(n_dim, 512, kernel_size=(5,5,1), stride=(2,2,1), padding=(2,2,1)),
            nn.LeakyReLU(inplace = True), #    nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(3,3,1), stride=(2,2,1)),

            nn.Conv3d(512, 256, kernel_size=(5,5,3), padding=(2,2,1)),
            nn.LeakyReLU(inplace = True), #    nn.ReLU(inplace=True),
            nn.Dropout(p = 0.8),
            nn.MaxPool3d(kernel_size=(3,3,1), stride=(2,2,1)),
            
            nn.BatchNorm3d(256),
            nn.Conv3d(256, 128, kernel_size=(5,5,3), padding=(2,2,1)),
            nn.LeakyReLU(inplace = True), #    nn.ReLU(inplace=True),
            nn.Dropout(p = 0.8),
            nn.MaxPool3d(kernel_size=(3,3,1), stride=(2,2,1)),
            
            nn.BatchNorm3d(128),
            nn.Conv3d(128, 384, kernel_size=(3,3,3), padding=(1,1,1)),
            nn.LeakyReLU(inplace = True), #    nn.ReLU(inplace=True),
           
            nn.BatchNorm3d(384),
            nn.Conv3d(384, 256, kernel_size=(3,3,3), padding=(1,1,1)),
            nn.LeakyReLU(inplace = True), #    nn.ReLU(inplace=True),
            
            nn.BatchNorm3d(256),
            nn.Conv3d(256, 256, kernel_size=(3,3,3), padding=(1,1,1)),
            nn.LeakyReLU(inplace = True), #    nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(3,3,1), stride=(2,2,1)),
        )
        self.classifier = nn.Sequential(
            nn.Linear(11520, 4096), #6 * 6* 4, 4096),
            nn.LeakyReLU(inplace = True), #    nn.ReLU(inplace=True),
            nn.Linear(4096, 512),
            nn.LeakyReLU(inplace = True), #    nn.ReLU(inplace=True),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0),x.size(1)*x.size(2)*x.size(3)*x.size(4) ) #6 * 6 * 4)
        x = self.classifier(x)
        return x
    
    
def alexnet_3d(pretrained=False, progress=True, **kwargs):
    r"""3D AlexNet model architecture, adapted from https://github.com/Ontheroad123/ImageNet/blob/master/torch-alexnet-3D.py
    
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    model = AlexNet_3D(**kwargs)
    if pretrained:
        "currently no pretained weights for 3D Alexnet available"
        pass 
    return model

In [12]:
import fastai.vision.models as models
from fastai.vision import *

In [7]:
# a very basic 2d conv net
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(20, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(40000, 1000)
        self.fc2 = nn.Linear(1000, 2)
        self.softmax =  nn.LogSoftmax(dim=1)

    def forward(self, x):
        if x.device.type == 'cpu': x = x.cuda()
        out = self.layer1(x)
   #     print(out.shape)
        out = self.layer2(out)
   #     print(out.shape)
        out = out.reshape(out.size(0), -1)
   #     print(out.shape)
        out = self.drop_out(out)
 #       print(out.shape)
        out = self.fc1(out)
 #       print(out.shape)
        out = self.fc2(out)
        out = self.softmax(out)
        # print(out.shape)
        return out

In [8]:
# a basic alexnet, adapted for batchsize
class AlexNet_(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet_, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(20, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2))
        
        self.layer3 = nn.Sequential(   
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True))
        
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True))
        
        self.layer5 = nn.Sequential(    
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2))
        
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
        
        self.softmax =  nn.LogSoftmax(dim=1)


    def forward(self, x):
        if x.device.type == 'cpu': x = x.cuda()
   #     print(x.shape)
        x = self.layer1(x)
   #     print(x.shape)
        x = self.layer2(x)
   #     print(x.shape)
        x = self.layer3(x)
   #     print(x.shape)
        x = self.layer4(x)
   #     print(x.shape)
        x = self.layer5(x)
   #     print(x.shape)
        x = self.avgpool(x)
   #     print(x.shape)
        x = torch.flatten(x, 1)
   #     print(x.shape)
        x = self.classifier(x)
        
        x = self.softmax(x)

        return x